# (보관용) 기존 더미 데이터 생성 노트북

## 왜 보관하는가?
이 노트북에서 생성된 더미 데이터는 모델 실험 파이프라인 동작 확인에는 유용했지만,
모델 성능 평가에는 부적합한 구조적 문제가 있었다.

---

## 발견된 문제점

### 1) 클래스가 선형적으로 완전히 분리됨
위험군/정상군이 점수·행동 변수에서 명확히 분리되도록 생성되어
Logistic Regression 같은 단순 모델도 Accuracy/Recall 1.0이 쉽게 발생했다.

### 2) 교차(모순) 케이스가 부족함
현실 데이터에서 흔한 아래 케이스가 반영되지 않았다.
- 점수는 낮지만 참여/과제는 높은 학생
- 점수는 높지만 결석이 많은 학생
- 수행평가만 높고 지필평가가 낮은 학생 등

### 3) 테스트셋이 너무 작고 불균형함
테스트 샘플 수가 작아 성능이 과대평가될 수 있다.

---

## 개선 방향
새로운 `00_generate_dummy_dataset.ipynb`에서는 아래를 반영한다.

- 위험군/정상군 분포가 겹치도록 노이즈 추가
- 교차 패턴(모순 케이스) 명시적으로 생성
- 라벨을 단순 점수 임계값만으로 결정하지 않도록 설계
- 클래스 비율(정상/위험군)을 현실적으로 조정


# 00_generate_dummy_dataset.ipynb

이 노트북은 **최소성취수준 보장지도(최성보) 위험군 예측** 프로젝트를 위한 더미 데이터셋을 생성합니다.

## 목표
- **단일 스키마(컬럼 고정)**로 데이터를 관리합니다.
- 학기 중간/학기 말 여부는 **값의 결측(예: `final_score` = NaN)**으로 표현합니다.
- 이후 모델/전처리 단계에서 **결측 열/결측 값**을 예외 처리하여 유저가 CSV를 쉽게 만들 수 있도록 합니다.

## 생성 결과
- `data/dummy/dummy_full.csv` : 모든 값이 채워진 더미 데이터(학기 말 가정)
- `data/dummy/dummy_midterm_like.csv` : 학기 중간 가정 데이터(`final_score` 결측 + 전반적 수행/참여 낮음)
- `data/dummy/data_dictionary.csv` : 컬럼 설명(데이터 사전)


In [10]:
from __future__ import annotations

from pathlib import Path
import numpy as np
import pandas as pd

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

DATA_DIR = Path("../data/dummy")
DATA_DIR.mkdir(parents=True, exist_ok=True)

print("DATA_DIR:", DATA_DIR.resolve())


DATA_DIR: D:\workSpace\edutech-risk-prediction\data\dummy


## 1) 스키마(단일)

| column | description |
|---|---|
| student_id | 비식별 학생 ID |
| midterm_score | 중간고사 성적 |
| final_score | 기말고사 성적 (학기 중간이면 NaN 가능) |
| performance_score | 수행평가 성적 (학기 중간이면 낮거나 일부만 반영될 수 있음) |
| assignment_count | 과제 제출 횟수 |
| participation_level | 수업 참여도 (상/중/하) |
| question_count | 수업 중 질문 횟수 |
| night_study | 야간자율학습 참여 여부 (0/1) |
| absence_count | 결석 횟수 |
| behavior_score | 상벌점 |


In [11]:
import math

# 생성 규모 (필요하면 변경)
N_STUDENTS = 80

# 결석률 기준(단순화): 한 학기 수업 회수 가정
TOTAL_SESSIONS = 30
ABSENCE_THRESHOLD = math.ceil(TOTAL_SESSIONS / 3)  # 1/3 이상이면 위험(단순화)

ABSENCE_THRESHOLD


10

## 2) 유틸 함수

In [12]:
def participation_to_num(x: str) -> int:
    return {"하": 0, "중": 1, "상": 2}[x]

def generate_participation(n: int, p=(0.25, 0.55, 0.20)) -> np.ndarray:
    # p: (상, 중, 하)
    return np.random.choice(["상", "중", "하"], size=n, p=list(p))

def clip_round(x: np.ndarray, lo=0, hi=100, nd=1) -> np.ndarray:
    return np.clip(x, lo, hi).round(nd)


## 3) 더미 데이터 생성(학기 말: full)

약한 상관을 주입합니다.

- 과제 제출, 참여도, 질문, 야자 참여 ↑ → 점수 ↑
- 결석 ↑ → 점수 ↓
- 상벌점은 약한 상관(노이즈 큼)


In [13]:
# 공통 생성 요소
student_id = np.arange(1, N_STUDENTS + 1)

participation_full = generate_participation(N_STUDENTS, p=(0.30, 0.55, 0.15))
participation_num_full = np.vectorize(participation_to_num)(participation_full)

night_study_full = np.random.binomial(1, p=0.55, size=N_STUDENTS)

assignment_full = np.random.poisson(lam=3.2, size=N_STUDENTS)
assignment_full = np.clip(assignment_full, 0, 8)

question_full = np.random.poisson(lam=0.7 + 0.6 * participation_num_full, size=N_STUDENTS)
question_full = np.clip(question_full, 0, 10)

absence_full = np.random.poisson(lam=1.1, size=N_STUDENTS)
# 소수 결석 과다
heavy_absent_idx = np.random.choice(N_STUDENTS, size=max(4, N_STUDENTS//20), replace=False)
absence_full[heavy_absent_idx] += np.random.randint(5, 10, size=len(heavy_absent_idx))
absence_full = np.clip(absence_full, 0, TOTAL_SESSIONS)

behavior_full = np.random.normal(loc=0, scale=4.5, size=N_STUDENTS).round().astype(int)
behavior_full = np.clip(behavior_full, -10, 10)

latent_full = (
    0.22 * (assignment_full / 8)
  + 0.25 * (participation_num_full / 2)
  + 0.18 * (question_full / 10)
  + 0.20 * night_study_full
  - 0.25 * (absence_full / TOTAL_SESSIONS)
  + 0.05 * (behavior_full / 10)
)
latent_full = np.clip(latent_full + np.random.normal(0, 0.08, size=N_STUDENTS), 0, 1)

midterm_full = clip_round(45 + 55 * latent_full + np.random.normal(0, 9, size=N_STUDENTS))
final_full = clip_round(42 + 58 * latent_full + np.random.normal(0, 10, size=N_STUDENTS))
performance_full = clip_round(50 + 50 * latent_full + np.random.normal(0, 8, size=N_STUDENTS))

df_full = pd.DataFrame({
    "student_id": student_id.astype(int),
    "midterm_score": midterm_full,
    "final_score": final_full,
    "performance_score": performance_full,
    "assignment_count": assignment_full.astype(int),
    "participation_level": participation_full,
    "question_count": question_full.astype(int),
    "night_study": night_study_full.astype(int),
    "absence_count": absence_full.astype(int),
    "behavior_score": behavior_full.astype(int),
})

df_full.head()


,student_id,midterm_score,final_score,performance_score,assignment_count,participation_level,question_count,night_study,absence_count,behavior_score
0,1,39.7,62.7,62.8,4,중,1,0,1,1
1,2,46.5,69.1,62.6,3,하,1,0,1,4
2,3,64.1,89.0,71.9,2,중,1,1,1,2
3,4,51.0,48.5,78.1,3,중,0,1,3,1
4,5,72.4,67.3,82.4,3,상,2,1,0,-7


## 4) 더미 데이터 생성(학기 중간: midterm-like)

요구사항 반영:
- `final_score`는 **비워둠(NaN)**
- 수행평가/참여/과제 등 일부 값이 **전반적으로 낮음**
- 결석은 약간 높을 수 있음(완만하게)

> 스키마는 동일하게 유지합니다.


In [14]:
participation_mid = generate_participation(N_STUDENTS, p=(0.15, 0.55, 0.30))
participation_num_mid = np.vectorize(participation_to_num)(participation_mid)

night_study_mid = np.random.binomial(1, p=0.40, size=N_STUDENTS)

assignment_mid = np.random.poisson(lam=2.0, size=N_STUDENTS)
assignment_mid = np.clip(assignment_mid, 0, 8)

question_mid = np.random.poisson(lam=0.4 + 0.4 * participation_num_mid, size=N_STUDENTS)
question_mid = np.clip(question_mid, 0, 10)

absence_mid = np.random.poisson(lam=2.0, size=N_STUDENTS)
# 소수 결석 과다
heavy_absent_idx2 = np.random.choice(N_STUDENTS, size=max(5, N_STUDENTS//16), replace=False)
absence_mid[heavy_absent_idx2] += np.random.randint(4, 9, size=len(heavy_absent_idx2))
absence_mid = np.clip(absence_mid, 0, TOTAL_SESSIONS)

behavior_mid = np.random.normal(loc=-1, scale=5.0, size=N_STUDENTS).round().astype(int)
behavior_mid = np.clip(behavior_mid, -10, 10)

latent_mid = (
    0.20 * (assignment_mid / 8)
  + 0.22 * (participation_num_mid / 2)
  + 0.15 * (question_mid / 10)
  + 0.18 * night_study_mid
  - 0.30 * (absence_mid / TOTAL_SESSIONS)
  + 0.03 * (behavior_mid / 10)
)
latent_mid = np.clip(latent_mid + np.random.normal(0, 0.10, size=N_STUDENTS), 0, 1)

midterm_mid = clip_round(38 + 52 * latent_mid + np.random.normal(0, 10, size=N_STUDENTS))
final_mid = np.array([np.nan] * N_STUDENTS, dtype=float)  # 학기 중간이라 비움
performance_mid = clip_round(35 + 45 * latent_mid + np.random.normal(0, 9, size=N_STUDENTS))

df_midterm_like = pd.DataFrame({
    "student_id": student_id.astype(int),
    "midterm_score": midterm_mid,
    "final_score": final_mid,
    "performance_score": performance_mid,
    "assignment_count": assignment_mid.astype(int),
    "participation_level": participation_mid,
    "question_count": question_mid.astype(int),
    "night_study": night_study_mid.astype(int),
    "absence_count": absence_mid.astype(int),
    "behavior_score": behavior_mid.astype(int),
})

df_midterm_like.head()


,student_id,midterm_score,final_score,performance_score,assignment_count,participation_level,question_count,night_study,absence_count,behavior_score
0,1,43.7,NaN,57.5,3,상,0,0,2,-6
1,2,56.6,NaN,48.7,3,상,3,1,2,-3
2,3,52.2,NaN,68.9,2,상,0,0,5,6
3,4,66.5,NaN,50.0,2,하,0,1,2,-2
4,5,44.6,NaN,17.9,2,하,0,1,5,6


## 5) (선택) 라벨 생성 예시

실제 운영에서는 학기 말에 성취율이 확정됩니다.  
더미에서는 `final_score`가 NaN일 수 있으므로, **성취율 계산 시 사용 가능한 항목만 반영**하는 예시를 제공합니다.

- 사용할 수 있는 점수 컬럼들만 평균으로 합성
- 결석 조건은 단순화(결석>=1/3)

> 이 라벨 생성은 추후 `src/modeling.py`에서 동일 원리로 적용할 수 있습니다.


In [15]:
def compute_achievement_rate_row(row: pd.Series) -> float:
    # NaN이 아닌 점수만 사용해서 평균을 냄 (단순 예시)
    score_cols = ["midterm_score", "final_score", "performance_score"]
    vals = [row[c] for c in score_cols if pd.notna(row[c])]
    if len(vals) == 0:
        return np.nan
    return float(np.mean(vals))

def add_labels(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["achievement_rate"] = out.apply(compute_achievement_rate_row, axis=1).round(1)
    out["at_risk"] = ((out["achievement_rate"] < 40) | (out["absence_count"] >= ABSENCE_THRESHOLD)).astype(int)
    return out

df_full_labeled = add_labels(df_full)
df_mid_labeled = add_labels(df_midterm_like)

print("Full risk rate:", df_full_labeled["at_risk"].mean().round(3))
print("Midterm-like risk rate:", df_mid_labeled["at_risk"].mean().round(3))

df_mid_labeled.head()


Full risk rate: 0.025
Midterm-like risk rate: 0.212


,student_id,midterm_score,final_score,performance_score,assignment_count,participation_level,question_count,night_study,absence_count,behavior_score,achievement_rate,at_risk
0,1,43.7,NaN,57.5,3,상,0,0,2,-6,50.6,0
1,2,56.6,NaN,48.7,3,상,3,1,2,-3,52.6,0
2,3,52.2,NaN,68.9,2,상,0,0,5,6,60.6,0
3,4,66.5,NaN,50.0,2,하,0,1,2,-2,58.2,0
4,5,44.6,NaN,17.9,2,하,0,1,5,6,31.2,1


## 6) 저장

In [16]:
(DATA_DIR / "dummy_full.csv").write_text(df_full.to_csv(index=False, encoding="utf-8-sig"), encoding="utf-8-sig")
(DATA_DIR / "dummy_midterm_like.csv").write_text(df_midterm_like.to_csv(index=False, encoding="utf-8-sig"), encoding="utf-8-sig")

# 라벨 포함 버전도 저장(선택)
(DATA_DIR / "dummy_full_labeled.csv").write_text(df_full_labeled.to_csv(index=False, encoding="utf-8-sig"), encoding="utf-8-sig")
(DATA_DIR / "dummy_midterm_like_labeled.csv").write_text(df_mid_labeled.to_csv(index=False, encoding="utf-8-sig"), encoding="utf-8-sig")

print("saved:")
for name in ["dummy_full.csv", "dummy_midterm_like.csv", "dummy_full_labeled.csv", "dummy_midterm_like_labeled.csv"]:
    print("-", (DATA_DIR / name).resolve())


saved:
- D:\workSpace\edutech-risk-prediction\data\dummy\dummy_full.csv
- D:\workSpace\edutech-risk-prediction\data\dummy\dummy_midterm_like.csv
- D:\workSpace\edutech-risk-prediction\data\dummy\dummy_full_labeled.csv
- D:\workSpace\edutech-risk-prediction\data\dummy\dummy_midterm_like_labeled.csv


## 7) 데이터 사전 저장

In [17]:
data_dict = pd.DataFrame({
    "column": [
        "student_id","midterm_score","final_score","performance_score","assignment_count",
        "participation_level","question_count","night_study","absence_count","behavior_score",
        "achievement_rate","at_risk"
    ],
    "type": [
        "int","float","float (NaN allowed)","float","int",
        "category(str)","int","int(0/1)","int","int",
        "float (computed)","int (computed)"
    ],
    "description": [
        "비식별 학생 ID",
        "중간고사 성적(0~100)",
        "기말고사 성적(0~100), 학기 중간이면 NaN",
        "수행평가 성적(0~100)",
        "과제 제출 횟수(0~8 가정)",
        "수업 참여도(상/중/하)",
        "수업 중 질문 횟수",
        "야간자율학습 참여 여부(0/1)",
        "결석 횟수",
        "상벌점(-10~10)",
        "성취율(사용 가능한 점수만 평균, 예시)",
        "최성보 위험군(성취율<40 또는 결석>=1/3, 예시)"
    ]
})
data_dict.to_csv(DATA_DIR / "data_dictionary.csv", index=False, encoding="utf-8-sig")
data_dict


,column,type,description
0,student_id,int,비식별 학생 ID
1,midterm_score,float,중간고사 성적(0~100)
2,final_score,float (NaN allowed),"기말고사 성적(0~100), 학기 중간이면 NaN"
3,performance_score,float,수행평가 성적(0~100)
4,assignment_count,int,과제 제출 횟수(0~8 가정)
5,participation_level,category(str),수업 참여도(상/중/하)
6,question_count,int,수업 중 질문 횟수
7,night_study,int(0/1),야간자율학습 참여 여부(0/1)
8,absence_count,int,결석 횟수
9,behavior_score,int,상벌점(-10~10)


## 8) 빠른 확인

In [18]:
print("dummy_full.csv shape:", df_full.shape)
print("dummy_midterm_like.csv shape:", df_midterm_like.shape)

print("\nMissing(final_score) in dummy_full:", df_full["final_score"].isna().sum())
print("Missing(final_score) in dummy_midterm_like:", df_midterm_like["final_score"].isna().sum())

df_midterm_like.describe(include="all")


dummy_full.csv shape: (80, 10)
dummy_midterm_like.csv shape: (80, 10)

Missing(final_score) in dummy_full: 0
Missing(final_score) in dummy_midterm_like: 80


,student_id,midterm_score,final_score,performance_score,assignment_count,participation_level,question_count,night_study,absence_count,behavior_score
count,80.0000,80.000000,0.0,80.000000,80.000000,80,80.000000,80.000000,80.000000,80.000000
unique,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,중,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,42,NaN,NaN,NaN,NaN
mean,40.5000,49.617500,NaN,44.752500,1.862500,NaN,0.637500,0.387500,2.362500,-0.937500
std,23.2379,11.720098,NaN,10.343566,1.338275,NaN,0.830491,0.490253,1.910953,5.205508
min,1.0000,18.800000,NaN,17.500000,0.000000,NaN,0.000000,0.000000,0.000000,-10.000000
25%,20.7500,42.050000,NaN,38.350000,1.000000,NaN,0.000000,0.000000,1.000000,-5.000000
50%,40.5000,49.950000,NaN,46.250000,2.000000,NaN,0.000000,0.000000,2.000000,-0.500000
75%,60.2500,56.350000,NaN,51.500000,3.000000,NaN,1.000000,1.000000,3.000000,3.000000
